In [1]:
# Creates the folder kemenyGPU
!git clone https://github.com/noeliarico/kemenyGPU

fatal: destination path 'kemenyGPU' already exists and is not an empty directory.


In [1]:
from numba import cuda
import numpy as np
import math
import time

In [ ]:
# create an array of dimension factorial(n) x n (initially empty)
# to store in each i-th row the factorial representation of i
stride = 4000000
# to store the rankings and factorial representations
# factoradic = np.zeros((stride,n), dtype=np.int64)
# d_factoradic = cuda.to_device(factoradic)
d_factoradic = cuda.device_array((stride,n), dtype=np.uint8)
# to store the boolean and factorial representations
# alternatives = np.zeros((stride,n), dtype=np.bool_)
# d_alternatives = cuda.to_device(alternatives)
d_alternatives = cuda.device_array((stride,n), dtype=np.bool_)


threadsperblock = 256
blockspergrid = math.ceil(stride / threadsperblock)
logging.debug("threadsperblock = {}, blockspergrid = {}, total = {}".format(threadsperblock, blockspergrid, threadsperblock*blockspergrid))
best_dist = np.array([10000]) 
d_best_dist = cuda.to_device(best_dist)
best_ranking = np.array([0], dtype=np.uint64) 
d_best_ranking = cuda.to_device(best_ranking)
single = False
total = math.factorial(n)
start = time.time()
distance_from_ranking_to_profile_gpu[blockspergrid, threadsperblock](d_factoradic, d_alternatives, om, stride, total, d_best_dist, d_best_ranking)


d_best_dist.copy_to_host(best_dist)
d_best_ranking.copy_to_host(best_ranking)
end = time.time()
logging.debug("Execution time: {}".format(end-start))
logging.debug(best_dist)
logging.debug(best_ranking)